<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#消し残り不良率計算" data-toc-modified-id="消し残り不良率計算-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>消し残り不良率計算</a></span><ul class="toc-item"><li><span><a href="#予備検討" data-toc-modified-id="予備検討-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>予備検討</a></span></li></ul></li></ul></div>

# 消し残り不良率計算

## 予備検討

In [19]:
import numpy as np
from scipy.special import comb
from scipy.special import factorial

In [20]:
def get_ndup( lcomb ):
    a = np.bincount( lcomb )
    a= factorial(a, exact=True )
    return np.prod( a )

In [21]:
def get_ncomb( ndepth, nopen, lcomb ):
    if  len(lcomb) == 1 :
        return ndepth
    elif ndepth == 1:
        return 0    # depthが足りないことはあり得ない
    else:
        a = get_ncomb( ndepth-1, nopen-lcomb[0], lcomb[1:])
#        print( a )
        return ndepth * comb(nopen, lcomb[0], exact=True ) *  a

In [22]:
def rmdup_ncomb( ncomb, lcomb ):
    return ncomb // get_ndup( lcomb )

In [23]:
def get_lcomb( num, ndepth ):
    ret = [[num]]
    for i in range(1,num//2+1):
        ret = ret +[ [i] + lst for lst in get_lcomb(num-i, ndepth) ]
        ret = [ lst for lst in ret if len(lst) <= ndepth ] # depthより長いリストはあり得ない
    return ret

In [24]:
def rmdup_lcomb( lcomb, ndepth=0 ):
    ret = [ sorted(lst) for lst in lcomb ] # 要素の中身をsort
    ret = sorted( [ tuple(lst) for lst in ret ] ) # tupleにして要素をsort
    ret = set(ret) # uniq
#    ret = [ lst for lst in ret if len(lst) <= ndepth ] # depthより長いリストはあり得ない
    return ret

In [27]:
def get_all_comb( ndepth, nopen ):
    lcomb = get_lcomb(nopen, ndepth)
    lcomb = rmdup_lcomb(lcomb)
    res = {}
    for tpl in lcomb:
        ncomb = get_ncomb( ndepth, nopen, tpl )
        res[tpl] = rmdup_ncomb( ncomb, tpl ) 
    return res   

In [34]:
ndepth = 13
nopen = 26
res = get_all_comb( ndepth, nopen )
print( res )
print( "total = ", ndepth**nopen )
print( "sum   = ", np.sum(list(res.values())))

KeyboardInterrupt: 

In [10]:
len(res.keys())

7

In [57]:
ncomb1( 2, 3, [1,2]) / 2**3

0.75

In [149]:
nopen = 4
ncombs = np.zeros( 4 )
for i in range(1,)

    

In [60]:
ncomb1( 30, 3, [1,2] ) / 30**3

0.09666666666666666

In [61]:
(1/30) * (29/30)

0.03222222222222222